In [ ]:
!pip install wandb
!pip install albumentations
!pip install tensorflow --upgrade

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import cv2
import datetime
import pathlib
import io
import os
import time
import random
import albumentations as A


from PIL import Image
from keras.models import Model
from keras.applications import EfficientNetB0 , ResNet50,EfficientNetB7 , VGG19
from keras.layers import (Layer , GlobalAveragePooling2D,Activation,MaxPooling2D,Add,Conv2D,MaxPool2D,Dense,Flatten,InputLayer,BatchNormalization,Input,
                          Embedding,Permute,Dropout,RandomFlip,RandomRotation,LayerNormalization,MultiHeadAttention,
                          RandomContrast,Rescaling,Resizing,Reshape,Cropping2D)
from keras.losses import BinaryCrossentropy,CategoricalCrossentropy,SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import Accuracy,TopKCategoricalAccuracy,CategoricalAccuracy,SparseCategoricalAccuracy
from keras.optimizers import Adam, Adadelta
from keras.callbacks import Callback, CSVLogger,EarlyStopping,LearningRateScheduler,ModelCheckpoint,ReduceLROnPlateau
from keras.regularizers import L1,L2


import wandb
from wandb.keras import WandbCallback



In [ ]:
CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 224,
    "LEARNING_RATE": 0.001,
    "N_EPOCHS": 10,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 12,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["fresh_apple",
"fresh_banana",
"fresh_bitter_gourd",
"fresh_capsicum",
"fresh_orange",
"fresh_tomato",
"stale_apple",
"stale_banana",
"stale_bitter_gourd",
"stale_capsicum",
"stale_orange",
"stale_tomato"],
}

In [ ]:
data_directory = "/kaggle/input/fresh-and-stale-images-of-fruits-and-vegetables"


# Wandb Configuration

In [ ]:
!wandb login bc9bc1971b37981a823d4bee42f570b5ceca496e

In [ ]:
wandb.init(project="FreshandStaleImages", entity="selcukozdemir")

In [ ]:
wandb.config = CONFIGURATION

# Dataset Loading

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
def dataset_loading():
    train_dataset = keras.utils.image_dataset_from_directory(
    data_directory,
    labels = "inferred",
    label_mode="categorical",
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset="training"
)
    validation_dataset = keras.utils.image_dataset_from_directory(
    data_directory,
    labels = "inferred",
    label_mode="categorical",
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
    subset="validation",
    validation_split = 0.1
    
)
    test_dataset = keras.utils.image_dataset_from_directory(
    data_directory,
    labels = "inferred",
    label_mode="categorical",
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode="rgb",
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.1,
    subset="validation"
)
    print(train_dataset)
   
    print(validation_dataset)
    return train_dataset,validation_dataset,test_dataset

In [ ]:
train_dataset,validation_dataset,test_dataset = dataset_loading()

# Dataset Vis

In [ ]:
plt.figure(figsize=(12,12))

for images,labels in train_dataset.take(2):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(images[i]/255)
        plt.title(CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis = 0).numpy()])
        plt.axis("off")
    

# Data Augmentation

In [ ]:
augment_layers = keras.Sequential([
    Cropping2D(cropping=((0,100),(0,0)))
])
def augment_layer(image,label):
    return augment_layers(image,training = True), label

# Resize and Rescale

In [ ]:
resize_rescale_layers = tf.keras.Sequential([
    Resizing(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"]),
])
def resize_rescale_layer(image,label):
    return resize_rescale_layers(image,training = True), label

In [ ]:
def resize_rescale(image, label):
  return tf.image.resize(image, (CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"])), label

# Dataset Preparation

In [ ]:
train_dataset = (train_dataset.prefetch(tf.data.AUTOTUNE))
validation_dataset = (validation_dataset.prefetch(tf.data.AUTOTUNE))


In [ ]:

plt.figure(figsize=(12,12))

for images,labels in train_dataset.take(4):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(images[i]/255)
        plt.title(CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis = 0).numpy()])
        plt.axis("off")

    



In [ ]:

plt.figure(figsize=(12,12))

for images,labels in validation_dataset.take(4):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(images[i]/255)
        plt.title(CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis = 0).numpy()])
        plt.axis("off")
    




In [ ]:
train_dataset

In [ ]:
validation_dataset

# MODELING

## Main Model

In [ ]:
lenet_model = tf.keras.Sequential(
    [
    InputLayer(input_shape = (None, None, 3), ), #giriş katmanı 3 kanaldan oluşacak gelecek şekli belli değil
    
    resize_rescale_layers, 
    
    Conv2D(filters = CONFIGURATION["N_FILTERS"] , kernel_size = CONFIGURATION["KERNEL_SIZE"], strides = CONFIGURATION["N_STRIDES"] , padding='valid',
          activation = 'relu',kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    MaxPool2D (pool_size = CONFIGURATION["POOL_SIZE"], strides= CONFIGURATION["N_STRIDES"]*2),
    Dropout(rate = CONFIGURATION["DROPOUT_RATE"] ),

    Conv2D(filters = CONFIGURATION["N_FILTERS"]*2 + 4, kernel_size = CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid',
          activation = 'relu', kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    MaxPool2D (pool_size = CONFIGURATION["POOL_SIZE"], strides= CONFIGURATION["N_STRIDES"]*2),

    Flatten(),
    
    Dense( CONFIGURATION["N_DENSE_1"], activation = "relu", kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    Dropout(rate = CONFIGURATION["DROPOUT_RATE"]),
    
    Dense( CONFIGURATION['N_DENSE_2'], activation = "relu", kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),

    Dense(CONFIGURATION["NUM_CLASSES"], activation = "softmax"),

])

lenet_model.summary()

# Transfer Learning with EfficentNet

In [ ]:
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

def build_efficentB0(num_classes):
    inputs = Input(shape=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"],3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    model.trainable = False

    x = GlobalAveragePooling2D(name = "avg_pool")(model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    outputs = Dense(num_classes,activation="softmax",name = "pred")(x)

    model = Model(inputs,outputs,name = "EfficentNet")

    optimizer = Adam(learning_rate=0.00001)

    model.compile(
        optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

def build_efficentB7(num_classes):
    inputs = Input(shape=(CONFIGURATION["IM_SIZE"],CONFIGURATION["IM_SIZE"],3))
    model = EfficientNetB7(include_top=False, input_tensor=inputs, weights="imagenet")

    model.trainable = False

    x = GlobalAveragePooling2D(name = "avg_pool")(model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    outputs = Dense(num_classes,activation="softmax",name = "pred")(x)

    model = Model(inputs,outputs,name = "EfficentNet")

    optimizer = Adam(learning_rate=0.00001)

    model.compile(
        optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
def build_resnet50(num_classes):
    pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Önceden eğitilmiş modelin üstüne özel bir sınıflandırıcı ekleme
    model = keras.Sequential()
    model.add(pretrained_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))  # 4 sınıf

    optimizer = Adam(learning_rate=0.00001)

    model.compile(
        optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"]
    )
    

    
    return model

In [ ]:
def build_vgg19(num_classes):
    pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pretrained_model.trainable = True

    # Önceden eğitilmiş modelin üstüne özel bir sınıflandırıcı ekleme
    input = Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"],3))

    x = backbone(input, training = False)
    x = GlobalAveragePooling2D()(x)
    x = Dense( CONFIGURATION["N_DENSE_1"], activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Dense( CONFIGURATION["N_DENSE_2"], activation = "relu")(x)
    output = Dense( CONFIGURATION["NUM_CLASSES"], activation = "softmax")(x)
    model = Model(input, output)

    model.compile(optimizer=keras.optimizers.experimental.SGD(lr=0.000000001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

## Freezing Model

In [ ]:
def unfreeze_model(model, start_layer, end_layer):
    for layer in model.layers[start_layer:end_layer]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])





In [ ]:
checkpoint = ModelCheckpoint("fine_tuned_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [ ]:
model = build_efficentB0(CONFIGURATION["NUM_CLASSES"])


In [ ]:
unfreeze_model(model,-10,None)

In [ ]:
history = model.fit(train_dataset,validation_data=validation_dataset,epochs=8,callbacks = [WandbCallback(),checkpoint])

In [ ]:
model.save("efficent_net_fresh_stale.h5")
model.save("efficent_net_fresh_stale.keras")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
backbone.compile(optimizer="adam",loss=CategoricalCrossentropy(),metrics=Accuracy())

In [ ]:
backbone.fit(train_dataset,validation_data=validation_dataset,epochs=5)

In [ ]:
#backbone.trainable = False

In [ ]:
pretrained_model = tf.keras.Sequential([
    Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"],3)),
    backbone,
    GlobalAveragePooling2D(),
    Dense( CONFIGURATION["N_DENSE_1"], activation = "relu"),
    BatchNormalization(),
    Dense( CONFIGURATION["N_DENSE_2"], activation = "relu"),
    Dense( CONFIGURATION["NUM_CLASSES"], activation = "softmax"),
    
    ])
pretrained_model.summary()

In [ ]:

input = Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"],3))

x = backbone(input, training = False)
x = GlobalAveragePooling2D()(x)
x = Dense( CONFIGURATION["N_DENSE_1"], activation = "relu")(x)
x = BatchNormalization()(x)
x = Dense( CONFIGURATION["N_DENSE_2"], activation = "relu")(x)
output = Dense( CONFIGURATION["NUM_CLASSES"], activation = "softmax")(x)

finetuned_model = Model(input, output)

In [ ]:

finetuned_model.summary()

# Callbacks

In [ ]:
checkpoint_callback = ModelCheckpoint(
    'best_weights', 
    monitor='val_accuracy',
    mode = 'max',
    verbose=1, 
    save_best_only=True,
    
    
    )

In [ ]:
class LogConfMatrix(Callback):
  def on_epoch_end(self, epoch, logs):
    predicted = []
    labels = []

    for im, label in test_dataset:
      predicted.append(pretrained_model(im))
      labels.append(label.numpy())

    pred = np.concatenate([np.argmax(predicted[:-1], axis = -1).flatten(), np.argmax(predicted[-1], axis = -1).flatten()])
    lab = np.concatenate([np.argmax(labels[:-1], axis = -1).flatten(), np.argmax(labels[-1], axis = -1).flatten()])
    
    cm = wandb.plot.confusion_matrix(
        y_true=lab,
        preds=pred,
        class_names=CONFIGURATION["CLASS_NAMES"])
        
    wandb.log({"conf_mat": cm})

In [ ]:
class LogResultsTable(Callback):
  def on_epoch_end(self, epoch, logs):
    
    columns=["image", "Predicted", "Label"]
    
    val_table = wandb.Table(columns = columns)

    
    for im, label in validation_dataset.take(25):

      pred = CONFIGURATION["CLASS_NAMES"][tf.argmax(pretrained_model(im), axis = -1).numpy()[0]]
      label = CONFIGURATION["CLASS_NAMES"][tf.argmax(label, axis = -1).numpy()[0]]

      row = [wandb.Image(im), pred, label]
      
      val_table.add_data(*row)

      
    wandb.log({"Model Results" : val_table})


# Train

In [ ]:
loss_function = CategoricalCrossentropy()

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"), TopKCategoricalAccuracy(k=2, name = "top_k_accuracy")]

In [ ]:
finetuned_model.compile(
  optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]),
  loss = loss_function,
  metrics = metrics,
)

In [ ]:
pretrained_model.save("B:\Dosya\Kodlar\Sem-Images-Classification\models\eff_keras.h5")